In [25]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
import numpy as np


# mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))
geo = SplineGeometry()
geo.AddCircle((0,0),1,bc="circle")

mesh = Mesh(geo.GenerateMesh(maxh=0.05))
#mesh = Mesh(geo.GenerateMesh(maxh=0.01))

#sine_theta = sin(atan2(y, x))

Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [26]:
order = 3

V1 = H1(mesh, order=order, dirichlet="circle")
V2 = NormalFacetFESpace(mesh, order=order-1, dirichlet="circle")
V = V1*V2

w,what = V.TrialFunction()
v,vhat = V.TestFunction()

In [27]:
mesh.GetBoundaries()

('circle', 'circle', 'circle', 'circle')

In [28]:
n = specialcf.normal(2)
h = specialcf.mesh_size

def jumpdn(v,vhat):
    return n*(grad(v)-vhat)
def hesse(v):
    return v.Operator("hesse")
def hessenn(v):
    return InnerProduct(n, hesse(v)*n)

dS = dx(element_boundary=True)
a = BilinearForm(V)
a += InnerProduct (hesse(w), hesse(v)) * dx \
     - hessenn(w) * jumpdn(v,vhat) * dS \
     - hessenn(v) * jumpdn(w,what) * dS \
     + 3*order*order/h * jumpdn(w,what) * jumpdn(v,vhat) * dS
a.Assemble()

#f = LinearForm(1*v*dx).Assemble()
#f = LinearForm(0.000000001*v*dx).Assemble()
neuval = 0.1818
f = LinearForm(neuval*v*dS).Assemble()
#f = LinearForm(V.GetFE())
#Impose Neumann BC
#f += 0.3333*v*ds(definedon="circle")


In [29]:


u = GridFunction(V)
# Set Dirichlet data
u.components[0].Set(0,BND)
u.vec.data = a.mat.Inverse(V.FreeDofs()) * f.vec

Draw (u.components[0], mesh, "disp_DG")

#Why is grad undefined?
#Draw (grad (u.components[0]), mesh, "grad")
Draw (grad(u.components[0]), mesh, "grad")
Draw (hesse (u.components[0]), mesh, "hesse")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [30]:
vfes = VectorH1(mesh, order = 1)

gu = GridFunction(vfes)
gu.Set(grad(u.components[0]))
lu = GridFunction(V1)
lu.Set(div(gu))
glu = GridFunction(vfes)
glu.Set(grad(lu))

Draw (glu, mesh, "Laplacian")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [31]:
# K=-e^{-2f}*\Delta f

curvatureK = GridFunction(V1)

curvatureK.Set(-exp(-2*u.components[0])*lu)

Draw (curvatureK, mesh, "Gaussian Curvature")

#print(exp.__doc__)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [32]:
from  ngsolve.internal import SnapShot
#ngsolve.Redraw(blocking=True)
SnapShot("aaa.bmp")
